<a href="https://colab.research.google.com/github/ShotaArima/practice_natural_language/blob/main/chap.1-Document%20Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第1章 文書のベクトル化

## janomeによる単語分割

In [1]:
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 17.0 MB/s eta 0:00:00


In [2]:
import janome
from janome.tokenizer import Tokenizer

In [3]:
t = Tokenizer()

In [4]:
for token in t.tokenize('私は秋田犬が大好きです。'):
  print(token)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
秋田	名詞,固有名詞,地域,一般,*,*,秋田,アキタ,アキタ
犬	名詞,一般,*,*,*,*,犬,イヌ,イヌ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
大好き	名詞,形容動詞語幹,*,*,*,*,大好き,ダイスキ,ダイスキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [5]:
# wakati=Trueで分かち書きモード
for token in t.tokenize('私は秋田犬が大好きです。', wakati=True):
  print(token, end='/')

私/は/秋田/犬/が/大好き/です/。/

In [6]:
# 半角スペースで分割
for token in t.tokenize('私は秋田犬が大好きです。', wakati=True):
  print(token, end=' ')

私 は 秋田 犬 が 大好き です 。 

In [7]:
# リストに格納
words = [token for token in t.tokenize('私は秋田犬が好きです。', wakati=True)]
print(words)

['私', 'は', '秋田', '犬', 'が', '好き', 'です', '。']


## N-gram

In [8]:
# bi-gram
for i in range (len(words)-1):
    print(f"{i}:{words[i:i+2]}")

0:['私', 'は']
1:['は', '秋田']
2:['秋田', '犬']
3:['犬', 'が']
4:['が', '好き']
5:['好き', 'です']
6:['です', '。']


In [9]:
# n-gram
def get_word_n_grams(sentence, n):
    words = [token for token in t.tokenize(sentence, wakati=True)]
    result = []
    for index in range(len(words)):
        result.append(words[index: index+n])
        if index+n >= len(words):
            return result

In [10]:
input = 'すもももももももものうち'
print(get_word_n_grams(input, 3))

[['すもも', 'も', 'もも'], ['も', 'もも', 'も'], ['もも', 'も', 'もも'], ['も', 'もも', 'の'], ['もも', 'の', 'うち']]


### 文字のN-gram

In [11]:
def get_charactor_n_gram(sentence, n):
    result = []
    for index in range(len(sentence)):
        result.append(sentence[index: index+n])
        if index+n >= len(sentence):
            return result

In [12]:
input2 = '私は秋田犬が大好きだ。'
print(get_charactor_n_gram(input2, 2))

['私は', 'は秋', '秋田', '田犬', '犬が', 'が大', '大好', '好き', 'きだ', 'だ。']


## Bag-of-words

In [13]:
def tokenize(sentence):
    return [token for token in t.tokenize(sentence, wakati=True)]

In [14]:
words1 = '私は秋田犬が大好き。'
words2 = '私は犬が少し苦手。'

print(f"words1:{tokenize(words1)}")
print(f"words2:{tokenize(words2)}")

words1:['私', 'は', '秋田', '犬', 'が', '大好き', '。']
words2:['私', 'は', '犬', 'が', '少し', '苦手', '。']


In [15]:
# 文書ベクトルの作成
def get_bag_of_words(document):
    result_dict = {}
    words = tokenize(document)
    for word in words:
        if word not in result_dict:
            result_dict[word] = 1
        else:
            result_dict[word] += 1
    return result_dict

In [16]:
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'
document2 = '私は犬が少し苦手。'
print(f"document1:{get_bag_of_words(document1)}")
print(f"document2:{get_bag_of_words(document2)}")

document1:{'私': 2, 'は': 2, '秋田': 2, '犬': 2, 'が': 2, '大好き': 2, '。': 2}
document2:{'私': 1, 'は': 1, '犬': 1, 'が': 1, '少し': 1, '苦手': 1, '。': 1}


In [17]:
# 辞書
def make_dictinary(documents):
    result_dict = {}
    index = 1
    for doc in documents:
        words = tokenize(doc)
        for word in words:
            if word not in result_dict:
                result_dict[word] = index
                index+=1
    return result_dict

In [18]:
documents = [document1, document2]
dictionary = make_dictinary(documents)
print(f"dict:{dict}")

dict:<class 'dict'>


In [19]:
# 用例ベクトルの作成
def make_BOW_vectors(documents, dictionary):
    result_vectors = []
    for doc in documents:
        vec = {}
        words = tokenize(doc)
        for entry in dictionary:
            vec[dictionary[entry]]=0
        for word in words:
            vec[dictionary[word]] += 1
        result_vectors.append(vec)
    return result_vectors

In [20]:
vectors = make_BOW_vectors(documents, dictionary)
print(f"結果:{vectors}")

id_to_word = {v: k for k, v in dictionary.items()} # 辞書型を逆変換する
word_occurrences = []
for occurrence in vectors:
    word_dict = {}
    for id_num, count in occurrence.items():
        word = id_to_word[id_num]
        word_dict[word] = count
    word_occurrences.append(word_dict)
for occurrences in word_occurrences:
    print(f"結果:{occurrences}")

結果:[{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 0, 9: 0}, {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 0, 7: 1, 8: 1, 9: 1}]
結果:{'私': 2, 'は': 2, '秋田': 2, '犬': 2, 'が': 2, '大好き': 2, '。': 2, '少し': 0, '苦手': 0}
結果:{'私': 1, 'は': 1, '秋田': 0, '犬': 1, 'が': 1, '大好き': 0, '。': 1, '少し': 1, '苦手': 1}


## CountVectorizerモジュール
- scikit-learnのモジュール

In [21]:
# wakatiを使用して単語間に半角スペースを入れる
def make_corps(documents):
    result_corps = []
    for doc in documents:
        words = tokenize(doc)
        text=" ".join(words)
        result_corps.append(text)
    return result_corps

In [22]:
t  = Tokenizer()
document1 = '私は秋田犬が大好き。秋田犬は私が大好き。'
document2 = '私は犬が少し苦手。'
documents = [document1, document2]
corpuses = make_corps(documents)
for corpus in corpuses:
    print(corpus)

私 は 秋田 犬 が 大好き 。 秋田 犬 は 私 が 大好き 。
私 は 犬 が 少し 苦手 。


In [23]:
# CountVectorizerを使用する
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
X = vectorizer.fit_transform(corpuses)
print(vectorizer.get_feature_names_out(corpuses))

['が' 'は' '大好き' '少し' '犬' '私' '秋田' '苦手']


In [25]:
print(X.toarray())

[[2 2 2 0 2 2 2 0]
 [1 1 0 1 1 1 0 1]]


In [26]:
# bi-gramでの出現回数
vectorizer2 = CountVectorizer(token_pattern='(?u)\\b\\w+\\b', ngram_range=(2,2))
X2 = vectorizer2.fit_transform(corpuses)
print(f"vectorizerのbi-gram:\n{vectorizer2.get_feature_names_out(corpuses)}")
print(f"vectorizerのbi-gramの出現回数:\n{X2.toarray()}")

vectorizerのbi-gram:
['が 大好き' 'が 少し' 'は 犬' 'は 私' 'は 秋田' '大好き 秋田' '少し 苦手' '犬 が' '犬 は' '私 が'
 '私 は' '秋田 犬']
vectorizerのbi-gramの出現回数:
[[2 0 0 1 1 1 0 1 1 1 1 2]
 [0 1 1 0 0 0 1 1 0 0 1 0]]


## TF-IDF
- TFとIDFの掛け算


### TF (Term Frequency)
- 単語出現頻度

<方法>
- 出現回数を利用する方法

$$
tf_{i, j} = n_{i, j} \tag{1.1}
$$
単語 $w_{i}$の文書 $d_j$における出現回数

- 出現回数の比率

$$
tf_{i, j} = \cfrac{n_{i, j}}{\sum_k n_{i, j}} \tag{1.2}
$$
分子は、単語 $w_i$の文書 $d_j$における出現回数
分母は、その単語の全ての文書中における出現回数の総和


### IDF (Inverse Document Frequency)
- 単語の逆文書頻度

$$
idf_{i, j} = \log{\cfrac{|D|}{|d:d \owns w_i |}} \tag{1.3}
$$
分子 $|D|$は、コーパス中における総文書数

分母は、単語 $w_i$を含む文書数

In [27]:
# TfidfVectorizerモジュールによるTF-IDFの文書ベクトル
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
X = vectorizer.fit_transform(corpuses)
print(vectorizer.get_feature_names_out(corpuses))

['が' 'は' '大好き' '少し' '犬' '私' '秋田' '苦手']


In [38]:
print(X.toarray())

[[0.35464863 0.35464863 0.49844628 0.         0.35464863 0.35464863
  0.49844628 0.        ]
 [0.35464863 0.35464863 0.         0.49844628 0.35464863 0.35464863
  0.         0.49844628]]


In [40]:
# n-gramの場合
vectorizer_ngram = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b', ngram_range=(2, 2))
X_ngram = vectorizer_ngram.fit_transform(corpuses)
print(X_ngram.toarray())

[[0.53428425 0.         0.         0.26714212 0.26714212 0.26714212
  0.         0.19007382 0.26714212 0.26714212 0.19007382 0.53428425]
 [0.         0.49922133 0.49922133 0.         0.         0.
  0.49922133 0.35520009 0.         0.         0.35520009 0.        ]]


In [41]:
new_X = vectorizer.transform(new_corpus)

NameError: name 'new_corpus' is not defined